In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [2]:
# orignal data 
data0=pd.read_csv('Iris.csv')
#print(data0)
col=data0.columns
data0.drop(columns=[col[0],col[1],col[4]],inplace=True,axis=1)
data0.head()

,SepalWidthCm,PetalLengthCm,Species
0,3.5,1.4,Iris-setosa
1,3.0,1.4,Iris-setosa
2,3.2,1.3,Iris-setosa
3,3.1,1.5,Iris-setosa
4,3.6,1.4,Iris-setosa


In [3]:
#  mix data 
data=pd.read_csv('Iris.csv')
col=data.columns
data.drop(columns=[col[0],col[1],col[4]],inplace=True,axis=1)
col=data.columns
data=data.replace({col[0]:3.2,col[1]:1.4},np.nan)
data.head()

,SepalWidthCm,PetalLengthCm,Species
0,3.5,NaN,Iris-setosa
1,3.0,NaN,Iris-setosa
2,NaN,1.3,Iris-setosa
3,3.1,1.5,Iris-setosa
4,3.6,NaN,Iris-setosa


In [4]:
data=pd.read_csv('Iris.csv')
col=data.columns
data.drop(columns=[col[0],col[1],col[4]],inplace=True,axis=1)
col=data.columns
data=data.replace({col[0]:3.2,col[1]:1.4},np.nan)
data.head()

,SepalWidthCm,PetalLengthCm,Species
0,3.5,NaN,Iris-setosa
1,3.0,NaN,Iris-setosa
2,NaN,1.3,Iris-setosa
3,3.1,1.5,Iris-setosa
4,3.6,NaN,Iris-setosa


In [5]:
# data having only mis value
ndataf=data[data.isnull().values==True]
ndataf=ndataf.reset_index()
ndataf.drop("index",inplace=True,axis=1)
#data.values[0,:]
ndataf.head(15)

,SepalWidthCm,PetalLengthCm,Species
0,3.5,NaN,Iris-setosa
1,3.0,NaN,Iris-setosa
2,NaN,1.3,Iris-setosa
3,3.6,NaN,Iris-setosa
4,3.4,NaN,Iris-setosa
5,2.9,NaN,Iris-setosa
6,3.0,NaN,Iris-setosa
7,3.5,NaN,Iris-setosa
8,3.4,NaN,Iris-setosa
9,NaN,1.6,Iris-setosa


In [6]:

ndata2=ndataf.replace(to_replace=np.nan,value=0,regex=True)
ndata2.head()

,SepalWidthCm,PetalLengthCm,Species
0,3.5,0.0,Iris-setosa
1,3.0,0.0,Iris-setosa
2,0.0,1.3,Iris-setosa
3,3.6,0.0,Iris-setosa
4,3.4,0.0,Iris-setosa


In [7]:
ndata1=data.dropna(how="any")
ndata1=ndata1.reset_index()
ndata1.drop("index",inplace=True,axis=1)
#data.values[0,:]
ndata1.head(10)

,SepalWidthCm,PetalLengthCm,Species
0,3.1,1.5,Iris-setosa
1,3.9,1.7,Iris-setosa
2,3.4,1.5,Iris-setosa
3,3.1,1.5,Iris-setosa
4,3.7,1.5,Iris-setosa
5,3.4,1.6,Iris-setosa
6,3.0,1.1,Iris-setosa
7,4.0,1.2,Iris-setosa
8,4.4,1.5,Iris-setosa
9,3.9,1.3,Iris-setosa


In [8]:
from sklearn import preprocessing
e = preprocessing.LabelEncoder()
e.fit(data.iloc[:,-1].unique())
ndataf.iloc[:,-1]=e.transform(ndataf.iloc[:,-1])
ndata1.iloc[:,-1]=e.transform(ndata1.iloc[:,-1])
ndata2.iloc[:,-1]=e.transform(ndata2.iloc[:,-1])
ndata1.head()

,SepalWidthCm,PetalLengthCm,Species
0,3.1,1.5,0
1,3.9,1.7,0
2,3.4,1.5,0
3,3.1,1.5,0
4,3.7,1.5,0


In [9]:
def find_null_feature(dt):
    col=data.columns
    temp=dt.iloc[:][dt.iloc[:].isnull()==True]
    l=len(temp.index)
    c=temp.index[0]
    
    if c==col[0]:
        label=0
        feature1=1
        feature2=2
    elif c==col[1]:
        label=1
        feature1=0
        feature2=2
    elif c==col[2]:
        label=2
        feature1=0
        feature2=1
        
    return l,label,feature1,feature2
    
    
    

In [10]:
#from sklearn.ensemble import RandomForestClassifier
def make_dataset(ndata,ndata1,n,d):
    from sklearn.ensemble import RandomForestRegressor
    clf = RandomForestRegressor(n_estimators=n, max_depth=d,
                                  random_state=0)
    dd=ndata.copy()
    s=ndata.shape[0]
    ind=[]
    for i in range(s):
        sample=ndata.iloc[i,:]
        l,label,feature1,feature2=find_null_feature(sample)
        if l==2:
            ind.append(i)
            continue
        clf.fit(ndata1.iloc[:,[feature1,feature2]],ndata1.iloc[:,label])
        sample=np.asarray(ndata.iloc[i,[feature1,feature2]]).reshape(1,2)
        a=clf.predict(sample)
        dd.iloc[i,label]=a
    dd.drop(ind,inplace=True)
    dd=dd.reset_index()
    dd.drop(columns='index',inplace=True)
    return dd

In [11]:
from bokeh.layouts import column,row
from bokeh.models import CustomJS, ColumnDataSource, Slider
from bokeh.plotting import Figure, output_file, show
from bokeh.plotting import figure
from bokeh.models import HoverTool, BoxSelectTool
from bokeh.layouts import column, row, WidgetBox
from bokeh.application import Application

In [12]:
src2 = ColumnDataSource(data0)

In [13]:
def make_plot(ndata1,source):
    #src2 = ColumnDataSource(data0)
    
    #TOOLS = [BoxSelectTool(), HoverTool(tooltips=[('Species', '@Species'), 
     #                               ('SepalWidthCm', '@SepalWidthCm'),
      #                              ('PetalLengthCm', '@PetalLengthCm')],
       #                   mode='vline')]
    
    
    hover=HoverTool(tooltips=[('Species', '@Species'), 
                                    ('SepalWidth','@x' ),
                                    ('PetalLength','@y')])
    '''
    hover = HoverTool(
        tooltips="""
        <div>
            <span style="font-size: 17px; font-weight: bold;">@SepalWidthCm</span>
            <span style="font-size: 15px; color: #966;">[$index]</span>
        </div>
        <div>
            <span style="font-size: 15px;">Location</span>
            <span style="font-size: 10px; color: #696;">($x, $y)</span>
        </div>
        """
    )
    '''

    output_file("js_on_change.html")

   # x = ndata1.iloc[:,0]
    #y = ndata1.iloc[:,1]
    #z=  ndata1.iloc[:,2]
    #g = data.groupby("species")
    #source = ColumnDataSource(data=dict(x=x, y=y))

    #colormap = {'Iris-setosa': 'red', 'Iris-versicolor': 'green', 'Iris-virginica': 'blue'}
    colormap = {0: 'red', 1: 'green', 2: 'blue'}

    colors = [colormap[x] for x in ndata1['Species']]

    p = figure(title = "Iris Morphology")
    p.xaxis.axis_label = "SepalWidthCm"
    p.yaxis.axis_label = "PetalLengthCm"

    p.circle(ndata1["SepalWidthCm"],ndata1["PetalLengthCm"],color=colors, fill_alpha=0.2, size=10)#ndata1["SepalWidthCm"], ndata1["PetalLengthCm"],
    #output_file("iris.html", title="iris.py example")color=colors, fill_alpha=0.2
    
    
    #x = ndata2.iloc[:,0]
    #y = ndata2.iloc[:,1]
    #z=  ndata2.iloc[:,2]
    #g = data.groupby("species")
    #source = ColumnDataSource(data=dict(x=x,y=y))

    #colormap = {'Iris-setosa': 'black', 'Iris-versicolor': 'black', 'Iris-virginica': 'black'}color=colors,, fill_alpha=0.2
    colormap = {0: 'black', 1: 'black', 2: 'black'}

    #colors = [colormap[x] for x in ndata2['Species']]
    p.circle(x='x',y='y',source=source, size=10)#ndata2["SepalWidthCm"], ndata2["PetalLengthCm"],
    output_file("iris.html", title="iris.py example")
    
    p.add_tools(hover)
    #show(p)
    return p


In [23]:
# Update function takes three default parameters
ntree = Slider(start = 1, end = 30, 
                     step = 1, value = 20,
                     title = 'n_estimator')

ndepth= Slider(start = 1, end = 5, 
                     step = 1, value = 3,
                     title = 'tree_depth')
#nsam= Slider(start = 1, end = 30, 
#                     step = 1, value = 1,
#                     title = 'n_sam')

x=ndata2.iloc[:,0]
y=ndata2.iloc[:,1]
source=ColumnDataSource(data=dict(x=x,y=y))


def update(attr, old, new):
    # Get the list of carriers for the graph
    n=ntree.value
    d=ndepth.value
    #s=nsam.value
    #print('call')
    # Make a new dataset based on the selected carriers and the 
    # make_dataset function defined earlier
    #print(n,d)
    ndata2 = make_dataset(ndataf,ndata1,n=n,d=d)
                          

    # Update the source used the quad glpyhs
    #ndata2.data.update(nndata2.data)
    x = ndata2.iloc[:,0]
    y = ndata2.iloc[:,1]
    #z = new_src.values[:,2]
    source.data=dict(x=x,y=y)
    #print(ndata2)
    
ntree.on_change('value', update) 
ndepth.on_change('value', update)
#nsam.on_change('value', update)

#ndata2= make_dataset(ndata,ndata1,n=ntree.value,d=ndepth.value)


#print(ndata2)
    
p = make_plot(ndata1,source)
show(p)
#output_file(p, title='Bokeh Plot', mode='inline', root_dir=None)

In [20]:
from bokeh.io import curdoc
inputs = column(ntree,ndepth)
#show(p)
curdoc().add_root(row(inputs, p, width=800))
curdoc().title = "Sliders"
output_file('hist3.html')
show(p)
output_file('hist3.html', title='Bokeh Plot', mode='inline', root_dir=None)
#output_file(p, title='Bokeh Plot', mode='inline', root_dir=None)[source]

In [21]:
#from bokeh.io import output_file
#output_file('testdem.html', title='Bokeh Plot', mode='inline', root_dir=None)
#output_file('hist2.html')
#show(p)
#output_file('hist3.html', title='Bokeh Plot', mode='inline', root_dir=None)

In [17]:
""""# Put controls in a single element
controls = WidgetBox(ntree,ndepth)

# Create a row layout
layout = row(controls, p)
#app = Application(layout)
show(layout)"""

'"# Put controls in a single element\ncontrols = WidgetBox(ntree,ndepth)\n\n# Create a row layout\nlayout = row(controls, p)\n#app = Application(layout)\nshow(layout)'